In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ------------------------------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_list <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [3]:
dset_idx <- 3

In [4]:
p_thresh <- 0.05
lfc_thresh <- log2(2)
q_deg_thresh <- 0.05
q_anova_thresh <- 0.05
coxph_coeff_thresh <- 0.0
mi_thresh <- 0.0
consensus_thresh <- 0.0
consensus_n <- 5
hub_con_thresh <- 0

# Functions

In [5]:
matrisome_df <- rutils::load_matrisome_df(matrisome_list) %>%
    dplyr::select(gene_symbol, division, category)


get_consensus_col <- function(df, n, thresh = 0) {
    consensus_df <- df %>%
        dplyr::select(geneID, contains("mean")) %>%
        dplyr::mutate(consensus = rowSums(.[-1] > thresh) >= n)
    df %>% dplyr::mutate(consensus = consensus_df$consensus)
}

deg_meta <- function(df, lfc_thresh, q_thresh, n) {
    deg_df <- df %>%
        dplyr::filter(abs(lfc) > lfc_thresh, qval < q_thresh)
    list(
        n_deg = nrow(deg_df),
        deg_prop = nrow(deg_df) / n,
        n_up = nrow(deg_df %>% dplyr::filter(lfc > 0)),
        n_down = nrow(deg_df %>% dplyr::filter(lfc < 0)),
        genes = deg_df$geneID
    )
}

simple_test_meta <- function(df, q_thresh) {
    filt_df <- df %>%
        dplyr::filter(qval < q_thresh)
    list(n_sig = nrow(filt_df), genes = filt_df$geneID)
}

mi_meta <- function(df, pct_max_thresh) {
    ord_df <- mi_figo_results_df %>%
        dplyr::arrange(desc(mi_est)) %>%
        dplyr::mutate(pct_max = mi_est / first(mi_est) * 100) %>%
        dplyr::filter(pct_max > pct_max_thresh)
    list(
        n_mi = nrow(ord_df),
        genes = ord_df$geneID
    )
}

lr_l1_meta <- function(score_df, res_df, baseline_df) {
    consensus_genes <- res_df %>%
        dplyr::filter(consensus == TRUE) %>%
        pull(geneID)
    avg_score <- mean(score_df$ref_score)
    naive_pct_imp <- (avg_score - baseline_df$naive) / baseline_df$naive * 100
    mc_pct_imp <- (avg_score - baseline_df$mc) / baseline_df$mc * 100
    list(
        avg_score = avg_score,
        naive_pct_imp = naive_pct_imp,
        mc_pct_imp = mc_pct_imp,
        n_genes = length(consensus_genes),
        genes = consensus_genes
    )
}


-- Column specification ------------------------------------------------------------------------------------------------------------------------
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)




# Data

In [6]:
norm_counts_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/", "norm_counts.tsv"))
m_norm_counts_df <- norm_counts_df %>%
    dplyr::filter(geneID %in% matrisome_df$gene_symbol)
deseq_results_df <- read_tsv(paste0(dirs$analysis_dir, "/deg/", unified_dsets[dset_idx], "_DESeq_results.tsv")) %>%
    rename(base_mean = baseMean, lfc = log2FoldChange)
welch_anova_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_welch_anova_results.tsv"))
mi_figo_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_MI_figo_results.tsv")) %>%
    dplyr::rename(mi_est = MI_est_median)
cls_baselines_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "cls_baselines.tsv"))
f1_l1_lr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_l1_lr_ref_scores.tsv"))
f1_l1_lr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_l1_lr_results.tsv")) %>%
    get_consensus_col(n = consensus_n, thresh = consensus_thresh)


-- Column specification ------------------------------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character()
)
i Use `spec()` for the full column specifications.



-- Column specification ------------------------------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double(),
  qval = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  pval = col_double(),
  padj = col_double(),
  qval = col_double()
)



-- Column specification -----------------------------------------------------------------------------------------------------

# DGE analysis

## DEG

In [7]:
deg_meta_ls <- deg_meta(deseq_results_df, lfc_thresh, q_deg_thresh, nrow(norm_counts_df))
deg_meta_ls[1:4]

$n_deg
[1] 8229

$deg_prop
[1] 0.406531

$n_up
[1] 4862

$n_down
[1] 3367

## DEMG

In [8]:
m_deseq_results_df <- deseq_results_df %>%
    dplyr::filter(geneID %in% matrisome_df$gene_symbol)
demg_meta_ls <- deg_meta(m_deseq_results_df, lfc_thresh, q_deg_thresh, nrow(m_norm_counts_df))
demg_meta_ls[1:4]

$n_deg
[1] 618

$deg_prop
[1] 0.6130952

$n_up
[1] 317

$n_down
[1] 301

# Uni/multivariate analysis

## Welch ANOVA

In [9]:
anova_meta_ls <- simple_test_meta(welch_anova_results_df, q_anova_thresh)
anova_meta_ls

$n_sig
[1] 16

$genes
 [1] "MUC3A"     "ANGPTL2"   "IL7"       "SERPIND1"  "SERPINA3"  "TGM3"     
 [7] "FGF14"     "LGI2"      "GDF7"      "IHH"       "MEGF9"     "PCOLCE2"  
[13] "HHIP"      "SERPINA11" "TNFSF11"   "PLG"

## MI

In [10]:
mi_meta_ls <- mi_meta(mi_figo_results_df, 50)
mi_meta_ls

$n_mi
[1] 52

$genes
 [1] "PLXDC2"   "ANXA13"   "GPC6"     "CHRDL2"   "PPBP"     "CCL24"   
 [7] "ADAM8"    "CCL7"     "RELN"     "PLXNA2"   "FGF19"    "CTSA"    
[13] "ANGPT4"   "IFNA5"    "IL12B"    "CLEC4D"   "OPTC"     "S100A16" 
[19] "SCUBE1"   "CELA3B"   "CX3CL1"   "IFNA6"    "FCN3"     "LOXL3"   
[25] "PCOLCE"   "SPOCK1"   "CHRD"     "CHAD"     "CBLN3"    "VIT"     
[31] "ADAMTSL2" "IBSP"     "CXCL6"    "NTNG1"    "COL5A3"   "MFAP1"   
[37] "FBN2"     "S100P"    "SHH"      "VWA1"     "TGM6"     "FGF12"   
[43] "CST6"     "SEMA3F"   "CLEC9A"   "PLXDC1"   "SFRP2"    "CTGF"    
[49] "LRG1"     "LAMA4"    "NELL2"    "IFNW1"

# ML baselines

In [11]:
cls_baselines <- as.list(cls_baselines_df[dset_idx,])
names(cls_baselines) <- c("dataset", "naive", "mc", "n_obs")
f1_macro_majority_baseline <- cls_baselines$naive
f1_macro_mc_baseline <- cls_baselines$mc
cls_baselines

$dataset
[1] "unified_uterine_endometrial_data"

$naive
[1] 0.1738095

$mc
[1] 0.2482832

$n_obs
[1] 137

# LR (L1)

In [12]:
lr_l1_meta_ls <- lr_l1_meta(f1_l1_lr_scores_df, f1_l1_lr_results_df, cls_baselines)
lr_l1_meta_ls[1:4]

$avg_score
[1] 0.3540268

$naive_pct_imp
[1] 103.6867

$mc_pct_imp
[1] 42.58994

$n_genes
[1] 76

In [13]:
umsmg_gold <- read_lines(paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_umsmg_list.txt"))
umsmg_new <- union(union(lr_l1_meta_ls$genes, mi_meta_ls$genes), anova_meta_ls$genes)
length(umsmg_gold)
length(umsmg_new)
length(intersect(umsmg_new, umsmg_gold))

[1] 135

[1] 135

[1] 135